<a href="https://colab.research.google.com/github/jesus1080/CompuacionEvolutiva-/blob/master/FabricaPaneles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fabrica de paneles de madera, el objetivo es crear la mayor ganancia $ con el material disponible y teniendo en cuenta las restricciones de material para cada panel de madera.


Instalación de la libreria deap

In [ ]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importar librerias requeridas

In [ ]:
import array
import random

import numpy

from deap import algorithms
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools


In [ ]:
IND_SIZE = 8
MIN_VALUE = 0
MAX_VALUE = 3000
MIN_STRATEGY = 250
MAX_STRATEGY = 1500

La función de inicialización para  estrategia de evolutiva no está definida por DEAP, por esto es necesario definirla.\

icls: la clase de individuo a instanciar.\
sclc: lase de estrategia para usar como estrategia.\
imin, imax, smin, smax: valores mínimo y máximo para el individuo y la estrategia.\

In [ ]:
# # Modifique esta funcion para generar individuos con estrategia de tipo entero
# Individual generator
def generateES(icls, scls, size, imin, imax, smin, smax):
    ind = icls(int(random.uniform(imin, imax)) for _ in range(size))
    ind.strategy = scls(int(random.uniform(smin, smax)) for _ in range(size))
    return ind


In [ ]:
# # Modifique el valor typecode de 'd' a 'I' para fijar individuos y estrategias de tipo entero positivo
creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode="I", fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="I")


In [ ]:
# # Funcion a maximizar de acuerdo al problema de paneles propuesto
import numpy as np

#retorna cero si el material para producir los paneles sobrepasan el material disponible en caso contrario retorna ganancia en dolaretes
def gananciaTotal(individuo):
  # Definir el array de 4 elementos

  ganancia = np.array([450, 1150, 800, 400])
  individuo = np.array(individuo[:4])

  # cada fila corresponde al gasto de material por unidad de panel, ejemplo para el panel Tahoe se gasta [ 1galon, 6minutos, 0pino, 0roble]
  matrizProduccion = np.array([[1, 6, 0, 0],
                              [1, 18, 8, 15],
                              [2, 12, 6, 6],
                              [1, 6, 4, 10]])

  # Cada fila representa material total para el individuo, ejemplo para galones de pegamentos [2,3,8,5]
  matrizTol = individuo * matrizProduccion.T

  suma = np.sum(matrizTol, axis=1)

  # Suma de cada fila corresponde a cada material pegamento, prensado, pino, roble

  #la funcion requiere devolver una tupla del size del individuo y el valor de aptitud

  b = len(individuo)
  s = np.dot(individuo, ganancia)


  #TODO verificar que cada material no exceda el limite y multiplicar el individuo por cada ganancia, sumar esa ganancia y retornar
  if suma[0]>5800 or suma[1]>43800 or suma[2]>29200 or suma[3]>60500:
    return 0,
  else:
    return s,

In [ ]:
toolbox = base.Toolbox()
toolbox.register("individual", generateES, creator.Individual, creator.Strategy,
    IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRATEGY, MAX_STRATEGY)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# # la funcion cxESBlend cruza individuos cuyos genes son valores continuos (float) se cambia a cxTwoPoint para numeros enteros
toolbox.register("mate", tools.cxTwoPoint)

# # para la mutacion mutESLogNormal realiza la mutación en los individuos de una población utilizando una distribución lognormal
# # se cambia a mutUniformInt  Esta función realiza una mutación uniforme en un rango de enteros especificado para cada gen del individuo.
toolbox.register("mutate", tools.mutUniformInt, low=10, up=50, indpb=0.03)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", gananciaTotal)



algorithms.eaMuCommaLambda: Este es el(μ+λ)algoritmo evolutivo
parametros:\
población – Una lista de individuos.\
caja de herramientas – A Toolboxque contiene los operadores de evolución.\
mu : el número de individuos a seleccionar para la próxima generación.\
lambda_ : el número de hijos a producir en cada generación.\
cxpb : la probabilidad de que se produzca una descendencia por cruzamiento.\
mutpb : la probabilidad de que se produzca una descendencia por mutación.\
ngen - El número de generación.\
stats : un Statisticsobjeto que se actualiza en el lugar, opcional.\
halloffame : un HallOfFameobjeto que contendrá a los mejores individuos, opcional.\
verbose : si registrar o no las estadísticas.



In [ ]:
def main():
    random.seed()
    MU, LAMBDA = 50, 80
    pop = toolbox.population(n=MU)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA,
        cxpb=0.6, mutpb=0.3, ngen=500, stats=stats, halloffame=hof)

    return pop, logbook, hof

if __name__ == "__main__":
    p,l,h=main()
    print(h[0])

gen	nevals	avg   	std   	min	max        
0  	50    	279924	773812	0  	2.85605e+06
1  	72    	934706	1.17897e+06	0  	2.9058e+06 
2  	71    	2.09863e+06	843295     	0  	2.9058e+06 
3  	72    	2.58561e+06	294110     	1.8706e+06	2.9058e+06 
4  	69    	2.8204e+06 	191919     	2.0757e+06	2.9615e+06 
5  	75    	2.9063e+06 	48274.7    	2.6898e+06	2.9615e+06 
6  	70    	2.92165e+06	20920.9    	2.9058e+06	2.9615e+06 
7  	76    	2.93555e+06	24997      	2.8578e+06	2.9703e+06 
8  	70    	2.95208e+06	16270.3    	2.9058e+06	2.9703e+06 
9  	68    	2.96034e+06	6675.93    	2.9282e+06	2.9703e+06 
10 	68    	2.9622e+06 	2387.38    	2.9615e+06	2.9703e+06 
11 	68    	2.96432e+06	4104.99    	2.9615e+06	2.9703e+06 
12 	75    	2.96889e+06	3226.13    	2.9615e+06	2.9703e+06 
13 	76    	2.9703e+06 	0          	2.9703e+06	2.9703e+06 
14 	76    	2.9703e+06 	0          	2.9703e+06	2.9703e+06 
15 	72    	2.9703e+06 	0          	2.9703e+06	2.9703e+06 
16 	72    	2.9703e+06 	0          	2.9703e+06	2.9703e+06 
17 	73   